# Calcularemos un dataset con las distancias entre las cadenas dentro de cada grupo isomorfo

In [1]:
!pip install levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [levenshtein]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [9]:
from Levenshtein import distance
import pandas as pd

df_groups = pd.read_csv("../BBDD_isomorphic_labeled_filtered_>5.csv", sep=";")
df_groups.head()


,wl_signature,sentences,Compleja,Compuesta,Compuesta-Compleja,Indeterminada (múltiples verbos sin subordinación/coordinación clara),Simple,Unimembre
0,"0_(1, 2):1|1_(0,):1|2_(0,):1","[10125, 2860, 3389, 8404, 3133, 1387, 3616, 80...",0,0,0,0,4,8
1,"0_(1,):1|1_(0, 2):1|2_(1, 4, 5):1|3_(4,):1|4_(...","[10009, 9687, 7799, 8368, 11181, 8011, 8452, 5...",0,0,0,0,7,1
2,"0_(1,):1|1_(0, 2):1|2_(1, 4, 6):1|3_(4,):1|4_(...","[2357, 5854, 743, 6089, 5679, 13880, 10386, 99...",1,0,0,1,7,0
3,"0_(1,):1|1_(0, 2):1|2_(1,):1","[5693, 1504, 9019, 5007, 479, 2189, 13538, 126...",0,0,0,1,6,8
4,"0_(1,):1|1_(0, 3):1|2_(3,):1|3_(1, 2, 4):1|4_(...","[6231, 2638, 6144, 8901, 11050, 5751, 5987, 11...",0,0,0,0,14,2


In [10]:
import networkx as nx
import itertools

def get_phrase(file):
    G = nx.read_graphml(file)
    return G.graph.get('phrase', 'No phrase found')

def get_phrases(id_list):
    phrases = {}
    for id in id_list:
        file = f"../UD_Spanish-GSD/es_gsd-ud-train_{id}.graphml"
        phrase = get_phrase(file)
        phrases[id] = phrase
    return phrases

def compute_distances(phrases):
    max_len = max(len(p) for p in phrases.values())
    distances = {}
    for (id_1, phrase_1), (id_2, phrase_2) in itertools.combinations(phrases.items(), 2):
        dist = distance(phrase_1, phrase_2) / max_len
        distances[(id_1, id_2)] = dist
    return distances

In [11]:
from ast import literal_eval

distances_by_group = {}

for index, row in df_groups.iterrows():
    id_list = literal_eval(row['sentences'])
    phrases = get_phrases(id_list)
    distances = compute_distances(phrases)
    distances_by_group[row["wl_signature"]] = distances

print(distances_by_group)

{'0_(1, 2):1|1_(0,):1|2_(0,):1': {(10125, 2860): 0.6666666666666666, (10125, 3389): 0.6666666666666666, (10125, 8404): 0.4583333333333333, (10125, 3133): 0.375, (10125, 1387): 0.4583333333333333, (10125, 3616): 0.7083333333333334, (10125, 8057): 0.5, (10125, 6255): 0.7916666666666666, (10125, 4202): 0.2916666666666667, (10125, 8540): 0.375, (10125, 8687): 0.4166666666666667, (2860, 3389): 0.75, (2860, 8404): 0.8333333333333334, (2860, 3133): 0.75, (2860, 1387): 0.6666666666666666, (2860, 3616): 0.5416666666666666, (2860, 8057): 0.7916666666666666, (2860, 6255): 0.7916666666666666, (2860, 4202): 0.75, (2860, 8540): 0.8333333333333334, (2860, 8687): 0.6666666666666666, (3389, 8404): 0.5416666666666666, (3389, 3133): 0.5833333333333334, (3389, 1387): 0.5833333333333334, (3389, 3616): 0.7916666666666666, (3389, 8057): 0.5833333333333334, (3389, 6255): 0.7916666666666666, (3389, 4202): 0.5416666666666666, (3389, 8540): 0.625, (3389, 8687): 0.625, (8404, 3133): 0.4583333333333333, (8404, 138

# Guardamos las distancias

In [22]:
def ids_without_close_neighbors(dist, threshold, assume_missing=None, ensure_nonempty=True):
    """
    Return the set of IDs that have no distance < threshold to any other ID.

    Parameters
    ----------
    dist : mapping from (id1, id2) -> distance
        Keys may be ordered or unordered; the function checks both (i, j) and (j, i).
    threshold : float
        Cutoff; IDs must be at least this far from everyone else.
    assume_missing : float or None
        If None, raise KeyError when a pair distance is missing.
        If a float (e.g., float('inf')), treat missing distances as that value.

    Example
    -------
    >>> d = {('A','B'):0.3, ('A','C'):0.6, ('B','C'):0.8}
    >>> ids_without_close_neighbors(d, 0.5)
    {'C'}
    """
    # collect all unique IDs present in keys
    ids = {x for pair in dist for x in pair}

    def get_distance(a, b):
        if a == b:
            return 0.0
        if (a, b) in dist:
            return dist[(a, b)]
        if (b, a) in dist:
            return dist[(b, a)]
        if assume_missing is None:
            raise KeyError(f"Missing distance for pair ({a}, {b}) and ({b}, {a}).")
        return assume_missing

    # keep those i whose distances to all others are >= threshold
    result = {i for i in ids if all(get_distance(i, j) >= threshold for j in ids if j != i)}
    if result or not ensure_nonempty:
        return result
    
    else:
        return [list(ids)[0]]

final_phrases = {}
for sign, dist in distances_by_group.items():
    ids = ids_without_close_neighbors(dist, 0.5)
    final_phrases[sign] = list(ids)
print(final_phrases)


{'0_(1, 2):1|1_(0,):1|2_(0,):1': [3616, 2860, 6255, 8057, 3389], '0_(1,):1|1_(0, 2):1|2_(1, 4, 5):1|3_(4,):1|4_(2, 3):1|5_(2,):1': [8452, 7799, 5930, 8011, 11181, 9687], '0_(1,):1|1_(0, 2):1|2_(1, 4, 6):1|3_(4,):1|4_(2, 3, 5):1|5_(4,):1|6_(2,):1': [743, 6089, 5679, 10386, 2357], '0_(1,):1|1_(0, 2):1|2_(1,):1': [5763, 12615, 13197, 2670, 5392, 11161, 9019], '0_(1,):1|1_(0, 3):1|2_(3,):1|3_(1, 2, 4):1|4_(3,):1': [6144, 5987, 7523, 8901, 10799, 12790, 5751, 6231, 4633, 2491, 11965], '0_(1,):1|1_(0, 3):1|2_(3,):1|3_(1, 2, 5, 6):1|4_(5,):1|5_(3, 4):1|6_(3,):1': [10787, 9831, 12363, 6815, 817, 151, 13689, 10106, 9469, 7487], '0_(1,):1|1_(0, 3, 4):1|2_(3,):1|3_(1, 2):1|4_(1,):1': [5475, 13797, 3887, 4049, 1241], '0_(1,):1|1_(0, 3, 5):1|2_(3,):1|3_(1, 2, 4):1|4_(3,):1|5_(1,):1': [9840, 1034, 12644, 1141], '0_(1,):1|1_(0, 3, 6):1|2_(3,):1|3_(1, 2, 5):1|4_(5,):1|5_(3, 4):1|6_(1,):1': [7809, 5060, 9879, 5654, 2550, 2073, 9791], '0_(1,):1|1_(0, 4):1|2_(4,):1|3_(4,):1|4_(1, 2, 3, 5):1|5_(4,):1': [3

In [23]:
import json, datetime


with open("final_groups.json", "w", encoding="utf-8") as f:
    json.dump(final_phrases, f, ensure_ascii=False, indent=2)